In [ ]:
import time
import os
from tqdm import tqdm

from main.request import Request
from main.itinerary import Itinerary
from main.scheduler import Scheduler
from main.database import Database
from main.insertion import Insertion
from main.leg import Leg
from main.scheduler import new_stop_from_stop, new_itinerary_from_itinerary
from main.globals import CONFIG_PATH

VERBOSE = 1

# Demand-responsive solver functions

## DR initialisation

In [ ]:
def request_from_db(database):
    """
    Creation of Request objects from customer information in the configuration file
    """
    db = database
    customers = db.get_customers()
    requests = []
    for customer in customers:
        customer_id = customer
        passenger_id = customer_id
        attributes = db.get_customer_dic(passenger_id)

        coords = db.get_customer_origin(customer_id)
        origin_id = db.get_stop_id([coords[1], coords[0]])

        coords = db.get_customer_destination(customer_id)
        destination_id = db.get_stop_id([coords[1], coords[0]])

        req = Request(db, passenger_id, origin_id, destination_id,
                              attributes.get("origin_time_ini"), attributes.get("origin_time_end"),
                              attributes.get("destination_time_ini"), attributes.get("destination_time_end"),
                              attributes.get("npass"))

        requests.append(req)
        if VERBOSE > 1:
            print("Created request from configuration file:")
            print(req.to_string())
    return requests

def itinerary_from_db(database):
    """
    Creation of initial Itinerary objects from vehicle information in the configuration file.
    Initial itineraries contain as first and last stop the warehouse where the vehicle is stored.

    Initialization of itinerary_insertion_dic, a data structure reflecting the insertions contained in each itinerary.
    """
    db = database
    transports = db.get_transports()
    itineraries = []
    itinerary_insertion_dic = {}
    for transport in transports:
        vehicle_id = transport

        coords = db.get_transport_origin(vehicle_id)
        start_stop_id = db.get_stop_id([coords[1], coords[0]])

        coords = db.get_transport_destination(vehicle_id)
        end_stop_id = db.get_stop_id([coords[1], coords[0]])

        attributes = db.get_transport_dic(vehicle_id)

        I = Itinerary(db, vehicle_id, attributes.get("capacity"), start_stop_id, end_stop_id,
                                attributes.get("start_time"), attributes.get("end_time"))
        itineraries.append(I)
        itinerary_insertion_dic[vehicle_id] = []
        if VERBOSE > 1:
            print("Created itinerary from configuration file:")
            print(I.to_string())
            print(I.start_stop.to_string())
            print(I.end_stop.to_string())
    return itineraries, itinerary_insertion_dic

def list_files(directory):
    try:
        # List all files in the specified directory
        files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
        return files
    except Exception as e:
        print(f"Error: {e}")
        return []

In [ ]:
def create_legs_from_itinerary(I, database, verbose=0):
    legs = []
    prev = None

    for S in I.stop_list:
        # Get previous leg
        if len(legs) > 0:
            prev = legs[-1]

        # Get S and T
        T = S.snext
        # if S is not the last stop, T is not None
        if T is not None:
            # Leg creation with temporal and distance costs
            l = Leg(itinerary=I.vehicle_id,
                    origin_stop=S,
                    dest_stop=S.snext,
                    passenger_id=S.snext.passenger_id,
                    time_cost=S.leg_time,
                    dist_cost=database.get_route_distance_km(S.id,S.snext.id),
                    prev=prev, next=None)
            # update result
            legs.append(l)
        # S is the last stop
        else:
            l = Leg(itinerary=I.vehicle_id,
                    origin_stop=S,
                    dest_stop=None,
                    passenger_id="DEPOT",
                    time_cost=S.leg_time,
                    dist_cost=0,
                    prev=prev, # prev -> l connection
                    next=None)
            # update result
            legs.append(l)

        # if S is not the first stop, update prev leg
        if prev is not None:
            prev.set_next(l) # prev -> l connection
    count = 0
    if VERBOSE > 0:
        for leg in legs:
            if verbose > 0: print(f"{count:3d}: {leg.__str__()}")
            count += 1
    return legs

def compute_stop_cost_dict(legs, I):
    # Cost of a stop visit: cost of prev -> S + cost of S -> next
    stop_cost_dict = {}
    for i in range(len(I.stop_list)):
        stop = I.stop_list[i]
        leg = legs[i]

        prev_dist_cost = 0
        next_dist_cost = 0
        prev_time_cost = 0
        next_time_cost = 0

        # not first stop
        if leg.prev is not None:
            prev_dist_cost = leg.prev.dist_cost
            prev_time_cost = leg.prev.time_cost

        # not last stop
        if leg.next is not None:
            next_dist_cost = leg.dist_cost
            next_time_cost = leg.time_cost

        stop_cost_dict[i] = { "stop_id": stop.id,
                             "leg": leg,
                             "dist_cost": prev_dist_cost+next_dist_cost,
                             "time_cost": prev_time_cost+next_time_cost}
    return stop_cost_dict

def get_customer_insertion(sche, vehicle_id, passenger_id):
    itinerary_insertions = sche.itinerary_insertion_dic.get(vehicle_id)
    for insertion in itinerary_insertions:
        if insertion.t.passenger_id == passenger_id:
            return insertion

## Database loading & instance solving

In [ ]:
def load_database():
    database = Database()
    return database

def load_and_solve_config(database, config_file):
    database.load_config(config_file)
    # Load itineraries from config file
    itineraries, itinerary_insertion_dic = itinerary_from_db(database)

    # Load requests from config file
    requests = request_from_db(database)

    # Create and initialize scheduler object
    sche = Scheduler(database)
    sche.pending_requests = requests
    sche.itineraries = itineraries
    sche.itinerary_insertion_dic = itinerary_insertion_dic
    # print("Solving config...")
    t1 = time.time()
    # Schedule all requests by order of issuance
    sche.schedule_all_requests_by_time_order(verbose=0)
    output = sche.simulation_stats()
    t2 = time.time()
    # print(f"Config solved, itineraries computed in {t2-t1:.2f} seconds")
    return sche, output

# Trip modification functions

## Modification creation with distance cost

In [ ]:
MINIMUM_COST_SAVE_KM = 0.1 # 100 meters
def compute_all_modifications(db, itineraries, verbose=0):
    # db = database
    itinerary_modification_dict = {}

    for itinerary in itineraries:
        itinerary_modification_dict[itinerary.vehicle_id] = None

    for I in itineraries:
        # Initialise mod_per_request_dict
        mod_per_request_dict = {}
        for request_id in I.customer_dict.keys():
            mod_per_request_dict[request_id] = []

        # 1. Create leg from itinerary
        itinerary_id = I.vehicle_id
        legs = create_legs_from_itinerary(I, db)
        # 2. Evaluate legs, creating stop_cost_dict
        stop_cost_dict = compute_stop_cost_dict(legs, I)
        # 3. Obtain id_cost_list to process entries in descending cost order
        id_cost_list = []
        for key, value in stop_cost_dict.items():
            id_cost_list.append((key, value.get('dist_cost')))
        id_cost_list.sort(key=lambda x: x[1], reverse=True)
        # 4. Process all entries, obtaining the modifications for each of them
        for i in range(len(id_cost_list)):
            modification_list = []
            if verbose > 0: print()
            # Gets dict entry number that corresponds to the next stop with higher cost
            dic_entry = id_cost_list[i][0]
            if verbose > 0: print(f"Entry nº{dic_entry}")

            # Gets associated stop information from stop_cost_dict (id, leg, dist_cost, time_cost)
            stop_info = stop_cost_dict[dic_entry]
            if verbose > 0: print(f"Stop costs: ", stop_info)

            # Get leg corresponding to that stop and previous leg
            stop_id = stop_info['stop_id']
            leg = stop_info['leg']
            prev = leg.prev
            if verbose > 0: print("\tLeg 1:", prev.__str__())
            if verbose > 0: print("\tLeg 2:", leg.__str__())
            current_cost = stop_info['dist_cost']
            # Modification type: origin/destination (stop) to be modified
            mod_type = None

            # If the first/last leg has not been chosen
            if prev is not None and leg.next is not None:
                # Get customer/request info associated with the stop to be modified
                # Such a customer is leg.prev.passenger_id
                customer_id = leg.prev.passenger_id
                # customer origin stop id
                origin_coords = db.get_customer_origin(leg.prev.passenger_id)
                origin_id = db.get_stop_id([origin_coords[1], origin_coords[0]])
                # customer destination stop id
                destination_coords = db.get_customer_destination(leg.prev.passenger_id)
                destination_id = db.get_stop_id([destination_coords[1], destination_coords[0]])
                if verbose > 0:
                    print(f"\t\tTrip to modify: Customer {customer_id} trip: {int(origin_id):3d} --> {int(destination_id):3d}")
                # the stop to be modified is the customer's origin
                if origin_id == stop_id:
                    mod_type = "origin"
                # the stop to be modified is the customer's destination
                elif destination_id == stop_id:
                    mod_type = "destination"
                else:
                    print(f"ERROR :: Stop to be modified {stop_id}, origin_id {origin_id}, destination_id {destination_id}")
                    exit()

                # Search for alternative stop among neighbours
                alter = False
                neighbours = db.get_neighbouring_stops_dict(stop_id, max_distance_km=2, geodesic=True)
                if len(neighbours) > 0:
                    alter = True
                    # Sort neighbours according to distance to the stop
                    neighbours.sort(key=lambda x: x[1], reverse=False)

                # If there are neighbouring stops
                if alter:
                    modifications = []
                    for j in range(len(neighbours)):
                        # Calculate costs if visiting each neighbour instead
                        # Select next neighbour
                        neigh_id = neighbours[j][0]
                        neigh_distance = neighbours[j][1]
                        # Avoid modifications that propose changing a customer origin with their destination and vice versa
                        if (mod_type == "origin" and neigh_id == destination_id) or (mod_type == "destination" and neigh_id == origin_id):
                            continue

                        # # TIME Cost of a stop visit: cost of prev -> S + cost of S -> next
                        # new_prev_time_cost = db.get_route_time_min(prev.origin_stop.id, neigh_id)
                        # new_next_time_cost = db.get_route_time_min(neigh_id, leg.dest_stop.id)
                        # new_time_cost = new_prev_time_cost + new_next_time_cost
                        # # Cost saving
                        # time_cost_save = current_cost - new_time_cost

                        # DISTANCE cost of a stop visit: cost of prev -> S + cost of S -> next
                        new_prev_dist_cost = db.get_route_distance_km(prev.origin_stop.id, neigh_id)
                        new_next_dist_cost = db.get_route_distance_km(neigh_id, leg.dest_stop.id)
                        new_dist_cost = new_prev_dist_cost + new_next_dist_cost
                        dist_cost_save = current_cost - new_dist_cost

                        # If the cost is improved
                        if dist_cost_save > MINIMUM_COST_SAVE_KM: # Distance cost
                            # Store stop modification proposal
                            mod = {
                                "vehicle_id": itinerary_id,
                                "customer_id": customer_id,
                                "mod_type": mod_type,
                                "stop_id": stop_id,
                                "neigh_id": neigh_id,
                                "neigh_distance": neigh_distance,
                                "cost_save": dist_cost_save,
                                # Ratio between saved distance and walking effort of the customers.
                                # if >1 the vehicle saves more distance than the customers have to walk.
                                "walk_to_save_ratio": dist_cost_save/neigh_distance
                            }
                            modifications.append(mod)
                    if len(modifications) > 0:
                        modifications.sort(key=lambda x: x["cost_save"], reverse=True)
                        # 5. Store modifications in list, sort by cost_save or ratio
                        mod_per_request_dict[customer_id] += modifications
                        if verbose > 0: print(f"\t\t\tRESULT :: {len(modifications)} modifications found")
                        if verbose > 0: print(f"\t\t\t\tMODIFICATIONS :: ")
                        if verbose > 0:
                            for mod_dict in modifications:
                                print(f"\t\t\t\t{mod_dict}")
                    else:
                        if verbose > 0:
                            print(f"\t\t\tRESULT :: No modifications for stop {stop_id} with current parameters.")
                else:
                    if verbose > 0:
                        print(f"\t\t\tWARNING :: No neighbours for stop {stop_id}. Skipping.")
            # If first/last leg have been chosen, skip
            else:
                if verbose > 0:
                    print(f"\t\t\tWARNING :: Candidate stop corresponds to DEPOT which cannot be modified. Skipping.")
        # end of THIS itinerary for
        # Sort modifications by cost_save or ratio
        for key in mod_per_request_dict.keys():
            mod_per_request_dict[key].sort(key=lambda x: x["cost_save"], reverse=True)
            # modification_list.sort(key=lambda x: x["cost_save"], reverse=True)
        # 6. Store list in dict
        itinerary_modification_dict[itinerary_id] = mod_per_request_dict

    return itinerary_modification_dict

In [ ]:
# Analise results
def print_modifications(itinerary_modification_dict):
    for key, value in itinerary_modification_dict.items():
        print(f"Itinerary {key} has modifications for {len(value)} requests")
        mod_per_request_dict = value
        print(mod_per_request_dict)
        for key, value in mod_per_request_dict.items():
            print(f"\tRequest {key} has {len(value)} modifications")
            if len(value) > 0:
                print(f"\t\tBest modification: {value[0]}")
        print()

In [ ]:
def get_best_mod_per_itinerary(itinerary_modification_dict):
    # Get best modifications for each request into a single itinerary-list
    all_mods_per_itinerary = {}
    for vehicle_id, value in itinerary_modification_dict.items():
        all_mods_per_itinerary[vehicle_id] = []
        mod_per_request_dict = value
        for request_id, value in mod_per_request_dict.items():
            if len(value) > 0:
                all_mods_per_itinerary[vehicle_id].append(value[0])

    for key in all_mods_per_itinerary.keys():
        all_mods_per_itinerary[key].sort(key=lambda x: x["cost_save"], reverse=True)

    return all_mods_per_itinerary

def get_best_modifications(itinerary_modification_dict, verbose=0):
    all_mods_per_itinerary = get_best_mod_per_itinerary(itinerary_modification_dict)
    # Get best modifications overall
    all_mods = []
    for vehicle_id in all_mods_per_itinerary.keys():
        all_mods += all_mods_per_itinerary[vehicle_id]

    all_mods.sort(key=lambda x: x["cost_save"], reverse=True)
    if verbose > 0:
        for entry in all_mods:
            print(f"\t{entry['vehicle_id']:20s}, {entry['customer_id']}, {entry['cost_save']:3.2f}km, (customer walks {entry['neigh_distance']*1000:3.2f} meters)")
    return all_mods

## Modification selection & feasibility check

In [ ]:
def get_next_best_modification(sche, all_mods, criteria, verbose=0):
    if criteria == "cost_save":
        all_mods.sort(key=lambda x: x["cost_save"], reverse=True)
    elif criteria == "walk_to_save_ratio":
        all_mods.sort(key=lambda x: x["walk_to_save_ratio"], reverse=True)
    elif criteria == "neigh_distance":
        all_mods.sort(key=lambda x: x["neigh_distance"], reverse=False)

    # Select the modification to implement
    mod = all_mods[0]
    vehicle_id = mod["vehicle_id"]
    passenger_id = mod["customer_id"]
    if verbose > 0: print(f"Modification to apply: {mod}")
    itinerary_to_modify = None
    for value in sche.itineraries:
        if value.vehicle_id == vehicle_id:
            itinerary_to_modify = value
    I = itinerary_to_modify
    if verbose > 0: print(f"Itinerary to modify:\n",I.to_string_simple())
    return mod, I

In [ ]:
def get_insertion_to_delete(sche, mod):
    vehicle_id = mod["vehicle_id"]
    passenger_id = mod["customer_id"]
    insertion_to_delete = get_customer_insertion(sche, vehicle_id, passenger_id)
    return insertion_to_delete

In [ ]:
def get_modified_trip_insertion(db, mod, insertion_to_delete, I):
    passenger_id = mod["customer_id"]
    # Get original customer request attributes
    attributes = db.get_customer_dic(passenger_id)

    # Current origin stop
    coords = db.get_customer_origin(passenger_id)
    origin_id = db.get_stop_id([coords[1], coords[0]])
    # Current destination stop
    coords = db.get_customer_destination(passenger_id)
    destination_id = db.get_stop_id([coords[1], coords[0]])

    # Get modified stop data
    if mod.get('mod_type') == 'origin':
        origin_id = mod.get('neigh_id')
        # origin_coords = db.get_stop_coords(origin_id)

    elif mod.get('mod_type') == 'destination':
        destination_id = mod.get('neigh_id')
        # destination_coords = db.get_stop_coords(destination_id)

    else:
        print(f"ERROR :: Mod type {mod.get('mod_type')} not recognized.")
        exit()


    # Update origin/destination stop id

    # Initialise request stops
    mod_Request = Request(db, passenger_id, origin_id, destination_id,
                          attributes.get("origin_time_ini"), attributes.get("origin_time_end"),
                          attributes.get("destination_time_ini"), attributes.get("destination_time_end"),
                          attributes.get("npass"))

    # Create insertion with same indexes as original one
    index_Spu = insertion_to_delete.index_Spu
    index_Ssd = insertion_to_delete.index_Ssd

    mod_insertion = Insertion(
        itinerary=I,
        trip=mod_Request,
        index_Spu=index_Spu,
        index_Ssd=index_Ssd,
        cost_increment=-1
    )
    return mod_insertion

## Modification implementation and cost_save retrieval

In [ ]:
def check_mod_trip_insertion_feasibility(I, mod_insertion, mod_request, verbose=0):
    # Check feasibility of inserting the modified customer trip in the same indexes of the itinerary

    # Extract Request's stops
    Spu = new_stop_from_stop(mod_request.Spu)
    if verbose > 0: print(f"Spu is {Spu.to_string_trip()}")
    Ssd = new_stop_from_stop(mod_request.Ssd)
    if verbose > 0: print(f"Ssd is {Ssd.to_string_trip()}")
    if verbose > 0: print()
    passenger_id = mod_request.passenger_id
    # Extract itinerary's stop list
    filtered_stops_i = I.stop_list

    # Feasibility of PICKUP stop
    index_Spu = mod_insertion.index_Spu
    # Index to insert pickup stop
    index_stop_i = index_Spu - 1
    try:
        R = filtered_stops_i[index_stop_i]
    except IndexError:
        print("ERROR Searching inside itinerary {}".format(I.vehicle_id))
        print(I.to_string())
        print()
        print("with the following list of filtered stops: {}".format([x.id for x in filtered_stops_i]))
        for x in filtered_stops_i:
            print(x.to_string())
        print()
        print("and an index_stop_i of: {}".format(index_stop_i))
        print(len(filtered_stops_i), index_stop_i)
        exit()

    T = R.snext
    if verbose > 0: print(f"R is {R.to_string()}")
    if verbose > 0: print(f"T is {T.to_string()}")
    # Check feasibility of inserting Spu in R's position, so that leg (R -> R.rnext)
    # becomes (Spu -> R.snext) therefore creating also a new leg (R -> Spu)
    test1, code = I.pickup_insertion_feasibility_check(mod_request, Spu, R, T)
    if verbose > 0: print(f"Spu insertion in position {index_stop_i+1}: {test1}\n")
    # Feasibility of SETDOWN stop
    index_Ssd = mod_insertion.index_Ssd
    if test1:
        # Once we select a feasible leg to insert Spu, store the index
        index_Spu = index_stop_i + 1
        # Copy of the itinerary to avoid modifications over the original
        I_with_Spu = new_itinerary_from_itinerary(I)
        # I_with_Spu = copy_Itinerary(I)
        # Insert Spu in the itinerary and re-calculate EAT carried forward over its putative successors
        I_with_Spu.insert_stop(Spu, index_Spu)
        # Compute the insertion's net additional cost
        I_with_Spu.compute_cost()
        if verbose > 0: print("Itinerary after insertion of Spu: {}\n".format(I_with_Spu.to_string_simple()))
        # Filter list of stops to keep only those not yet visited
        filtered_stops_j = [new_stop_from_stop(x) for x in I_with_Spu.stop_list]

        # Index to insert setdown stop
        index_stop_j = index_Ssd - 1
        R = filtered_stops_j[index_stop_j]
        T = R.snext
        if verbose > 0: print(f"\tR is {R.to_string()}")
        if verbose > 0: print(f"\tT is {T.to_string()}")
        test2, code = I_with_Spu.setdown_insertion_feasibility_check(mod_request, index_Spu,
                                                                     index_stop_j + 1,
                                                                     I_with_Spu.stop_list, Ssd, R, T)
        if verbose > 0: print(f"\tSsd insertion in position {index_stop_j+1}: {test2}")
        if test2:
            I_with_Spu_Ssd = new_itinerary_from_itinerary(I_with_Spu)
            I_with_Spu_Ssd.insert_stop(Ssd, index_Ssd)
            # Compute the insertion's net additional cost
            I_with_Spu_Ssd.compute_cost()
            if verbose > 0: print("Itinerary after insertion of Ssd: {}\n".format(I_with_Spu_Ssd.to_string_simple()))

            mod_insertion = Insertion(
                                itinerary=I,
                                trip=mod_request,
                                index_Spu=index_Spu,
                                index_Ssd=index_Ssd,
                                cost_increment=-1
                            )
            return True, mod_insertion
        else:
            # print(f"Test 2 failed:{code}")
            return False, None
    else:
        # print(f"Test 1 failed: {code}")
        return False, None

# Auxiliary functions

In [ ]:
def get_config_attributes(filename):
    first_r = filename.index('r')
    first_v = filename.index('v')

    val1 = filename[0:first_r]

    val2 = filename[first_v-2:first_v]
    return val1, val2

# script

In [ ]:
# Load database
database = load_database()

In [ ]:
config_files = list_files(CONFIG_PATH)
config_files = [x for x in config_files if x.endswith('.json')]
print(f"Instance, Scheduled (%), Cost, # F mod., # F req., Max. save (%), F save (%), Avg. save/req")

files_and_attributes = [(x, get_config_attributes(x)) for x in config_files]
files_and_attributes.sort(key=lambda x: sum([int(x[1][0]), int(x[1][1])]))

for config_file, attributes in files_and_attributes:
    # Compute initial itineraries
    sche, output = load_and_solve_config(database, config_file)

    # Compute all modifications, filter best
    itinerary_modification_dict = compute_all_modifications(database, sche.itineraries, verbose=0)

    # Prepare a list with all modifications
    total_mods = []
    for key in itinerary_modification_dict:
        request_dict = itinerary_modification_dict[key]
        for key in request_dict:
            mod_list = request_dict[key]
            for mod in mod_list:
                total_mods.append(mod)
    elegible_req = set([x["customer_id"] for x in total_mods])

    # Process modifications iteratively to assess feasibility
    feasible_mods = []
    unfeasible_mods = []
    for mod_index in range(len(total_mods)):
        mod = total_mods[mod_index]
        # Get the itinerary of the modification
        vehicle_id = mod["vehicle_id"]
        passenger_id = mod["customer_id"]

        # Get corresponding insertions
        insertion_to_delete = get_insertion_to_delete(sche, mod)
        itinerary_to_modify = insertion_to_delete.I
        aux_I = new_itinerary_from_itinerary(itinerary_to_modify)
        mod_insertion =  get_modified_trip_insertion(database, mod, insertion_to_delete, itinerary_to_modify)

        # Remove the trip
        sche.remove_trip(insertion_to_delete)

        # Check the feasibility of modified insertion
        feasible, mod_insertion = check_mod_trip_insertion_feasibility(itinerary_to_modify, mod_insertion, mod_insertion.t, verbose=0)
        if feasible:
            feasible_mods.append(mod)
        else:
            unfeasible_mods.append(mod)

        # Restore modified itinerary
        sche.insert_trip(insertion_to_delete)

    feasible_req = set([x["customer_id"] for x in feasible_mods])
    max_cost_save = sum([x["cost_save"] for x in feasible_mods])
    total_cost = output["total_cost"]

    best_mod_per_request_dict = {}
    for req in feasible_req:
        best_mod_per_request_dict[req] = []

    for mod in feasible_mods:
        best_mod_per_request_dict[mod["customer_id"]].append(mod)

    for key, value in best_mod_per_request_dict.items():
        value.sort(key=lambda x: x["cost_save"], reverse=True)

    applicable_mods = []
    for key, value in best_mod_per_request_dict.items():
        applicable_mods.append(value[0])
    feasible_cost_save = sum([x["cost_save"] for x in applicable_mods])

    # print(f"Instance, Scheduled (%), Cost, Mods., Elegible req., Feasible mods., Feasible req., Max. cost save (%), Feasible cost save (%)")
    print(f"{int(attributes[0]):3d}r-{int(attributes[1]):2d}v, "
          f"{output['scheduled_requests']:3d} ({output['scheduled_percent']:3.1f}), "
          f"{output['total_cost']:4.0f}, ",
          # f"{len(total_mods):3d}, ",
          # f"{len(elegible_req):3d}, ",
          f"{len(feasible_mods):3d}, ",
          f"{len(feasible_req):3d}, ",
          f"{max_cost_save:4.0f} ({(max_cost_save/total_cost)*100:3.1f}), ",
          f"{feasible_cost_save:4.0f} ({(feasible_cost_save/total_cost)*100:3.1f}), ",
          f"{feasible_cost_save/len(feasible_req):2.1f}")